In [12]:
!pip install dgl
!pip install torch==1.9.1

     |████████████████████████████████| 831.4 MB 6.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.1 which is incompatible.


In [7]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [8]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [10]:
from dgl.data import CoraGraphDataset
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask

In [11]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [12]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(epoch, loss.item(), acc, np.mean(dur)))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch 00000 | Loss 1.9653 | Test Acc 0.2740 | Time(s) nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00001 | Loss 1.8601 | Test Acc 0.2600 | Time(s) nan
Epoch 00002 | Loss 1.7160 | Test Acc 0.2440 | Time(s) nan
Epoch 00003 | Loss 1.5796 | Test Acc 0.3530 | Time(s) 0.0578
Epoch 00004 | Loss 1.4688 | Test Acc 0.5030 | Time(s) 0.0624
Epoch 00005 | Loss 1.3575 | Test Acc 0.6400 | Time(s) 0.0602
Epoch 00006 | Loss 1.2521 | Test Acc 0.7010 | Time(s) 0.0597
Epoch 00007 | Loss 1.1595 | Test Acc 0.7230 | Time(s) 0.0593
Epoch 00008 | Loss 1.0759 | Test Acc 0.7110 | Time(s) 0.0589
Epoch 00009 | Loss 0.9949 | Test Acc 0.7030 | Time(s) 0.0572
Epoch 00010 | Loss 0.9193 | Test Acc 0.7000 | Time(s) 0.0566
Epoch 00011 | Loss 0.8491 | Test Acc 0.7040 | Time(s) 0.0555
Epoch 00012 | Loss 0.7820 | Test Acc 0.7180 | Time(s) 0.0550
Epoch 00013 | Loss 0.7163 | Test Acc 0.7350 | Time(s) 0.0543
Epoch 00014 | Loss 0.6539 | Test Acc 0.7520 | Time(s) 0.0535
Epoch 00015 | Loss 0.5977 | Test Acc 0.7650 | Time(s) 0.0528
Epoch 00016 | Loss 0.5475 | Test Acc 0.7660 | Time(s) 0.0526
Epoch 00017 | Loss 0.5010 | Te